In [ ]:
!pip install torch torchvision opencv-python numpy pandas matplotlib scikit-learn torchsummary kagglehub

In [1]:
import os
import h5py
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import kagglehub
import glob
import random
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, auc, classification_report
from sklearn.preprocessing import label_binarize
from itertools import cycle

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from torch.optim.lr_scheduler import CosineAnnealingLR

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Download the dataset using kagglehub
print("Downloading dataset...")
dataset_path = kagglehub.dataset_download("ashkhagan/figshare-brain-tumor-dataset")
print("Path to dataset files:", dataset_path)

# Data loading functions
def load_data():
    """Load and preprocess data from the downloaded directory"""
    # First find all .mat files in the dataset directory and its subdirectories
    mat_files = []
    for root, dirs, files in os.walk(dataset_path):
        for file in files:
            if file.endswith('.mat') and file != 'cvind.mat':
                mat_files.append(os.path.join(root, file))
   
    print(f"Found {len(mat_files)} .mat files")
   
    # If we don't have enough files, exit
    if len(mat_files) < 3000:
        raise ValueError(f"Expected ~3064 .mat files but found only {len(mat_files)}")
   
    # Sort the files to ensure consistent order
    mat_files.sort()
   
    # Prepare arrays for images and labels
    img = np.zeros((len(mat_files), 224, 224))
    lbl = []
   
    # Load each file
    for i, file_path in enumerate(mat_files):
        try:
            with h5py.File(file_path, 'r') as f:
                images = f['cjdata']
                resized = cv2.resize(images['image'][:,:], (224, 224), interpolation=cv2.INTER_CUBIC)
                x = np.asarray(resized)
                x = (x - np.min(x)) / (np.max(x) - np.min(x))  # Normalization
                x = x.reshape((1, 224, 224))
                img[i] = x
                lbl.append(int(images['label'][0]))
               
                if i % 500 == 0:
                    print(f"Processed {i} images")
        except Exception as e:
            print(f"Failed to load image at {file_path}: {e}")
   
    # Find cvind.mat file
    cvind_files = []
    for root, dirs, files in os.walk(dataset_path):
        for file in files:
            if file == 'cvind.mat':
                cvind_files.append(os.path.join(root, file))
   
    if not cvind_files:
        raise ValueError("Could not find cvind.mat file")
   
    cvind_path = cvind_files[0]
    print(f"Found cvind.mat at: {cvind_path}")                                                              # Load fold indices
    with h5py.File(cvind_path, 'r') as f:
        idx = np.array(f['cvind']).astype(np.int16).squeeze()
   
    return img, np.array(lbl), idx

# Custom Dataset
class BrainTumorDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform
       
    def __len__(self):
        return len(self.labels)
   
    def __getitem__(self, idx):
        # Convert grayscale to RGB by repeating channel
        image = self.images[idx]
        image = np.repeat(image.reshape(224, 224, 1), 3, axis=2)
        label = self.labels[idx] - 1  # Convert to 0-indexed
       
        if self.transform:
            image = self.transform(image)
        else:
            image = torch.from_numpy(image.transpose(2, 0, 1)).float()
           
        return image, label

# Define transforms for training and validation
def get_transforms():
    train_transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.RandomRotation(15),
        transforms.RandomAffine(degrees=0, translate=(0.05, 0.05)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
   
    val_transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
   
    return train_transform, val_transform

# Get train and test splits
def get_train_test_data(images, labels, fold_indices, test_fold):
    train_mask = fold_indices != test_fold
    test_mask = fold_indices == test_fold
   
    train_images = images[train_mask]
    train_labels = labels[train_mask]
    test_images = images[test_mask]
    test_labels = labels[test_mask]
   
    return (train_images, train_labels), (test_images, test_labels)

# Squeeze and Excitation Block
class SEBlock(nn.Module):
    def __init__(self, channel, reduction=16):
        super(SEBlock, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
            nn.Linear(channel, channel // reduction, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(channel // reduction, channel, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        b, c, _, _ = x.size()
        y = self.avg_pool(x).view(b, c)
        y = self.fc(y).view(b, c, 1, 1)
        return x * y.expand_as(x)

# Feature Pyramid Network (FPN) module
class FPN(nn.Module):
    def __init__(self, channels):
        super(FPN, self).__init__()
        self.lateral_conv = nn.Conv2d(channels, 256, kernel_size=1)
        self.output_conv = nn.Conv2d(256, 256, kernel_size=3, padding=1)
       
    def forward(self, x):
        lateral = self.lateral_conv(x)
        output = self.output_conv(lateral)
        return output

# Transformer-based attention block
class TransformerBlock(nn.Module):
    def __init__(self, dim, num_heads=4, mlp_ratio=4.0, qkv_bias=True, dropout=0.1):
        super(TransformerBlock, self).__init__()
        self.norm1 = nn.LayerNorm(dim)
        self.attn = nn.MultiheadAttention(dim, num_heads, dropout=dropout, batch_first=True)
        self.norm2 = nn.LayerNorm(dim)
        self.mlp = nn.Sequential(
            nn.Linear(dim, int(dim * mlp_ratio)),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(int(dim * mlp_ratio), dim),
            nn.Dropout(dropout)
        )
       
    def forward(self, x):
        # Reshape to sequence for attention
        b, c, h, w = x.shape
        x_seq = x.flatten(2).transpose(1, 2)  # B, HW, C
       
        # Apply normalization
        x_norm = self.norm1(x_seq)
       
        # Self-attention
        attn_out, _ = self.attn(x_norm, x_norm, x_norm)
        x_seq = x_seq + attn_out
       
        # MLP block
        x_seq = x_seq + self.mlp(self.norm2(x_seq))
       
        # Reshape back to feature map
        x = x_seq.transpose(1, 2).reshape(b, c, h, w)
       
        return x

# Coordinate Attention
class CoordinateAttention(nn.Module):
    def __init__(self, inp, oup, reduction=32):
        super(CoordinateAttention, self).__init__()
        self.pool_h = nn.AdaptiveAvgPool2d((None, 1))
        self.pool_w = nn.AdaptiveAvgPool2d((1, None))
       
        mip = max(8, inp // reduction)
       
        self.conv1 = nn.Conv2d(inp, mip, kernel_size=1, stride=1, padding=0)
        self.bn1 = nn.BatchNorm2d(mip)
        self.act = nn.ReLU(inplace=True)
       
        self.conv_h = nn.Conv2d(mip, oup, kernel_size=1, stride=1, padding=0)
        self.conv_w = nn.Conv2d(mip, oup, kernel_size=1, stride=1, padding=0)
       
    def forward(self, x):
        identity = x
       
        n, c, h, w = x.size()
        x_h = self.pool_h(x)  # n,c,h,1
        x_w = self.pool_w(x).permute(0, 1, 3, 2)  # n,c,1,w -> n,c,w,1
       
        y = torch.cat([x_h, x_w], dim=2)
        y = self.conv1(y)
        y = self.bn1(y)
        y = self.act(y)
       
        x_h, x_w = torch.split(y, [h, w], dim=2)
        x_w = x_w.permute(0, 1, 3, 2)
       
        a_h = self.conv_h(x_h).sigmoid()
        a_w = self.conv_w(x_w).sigmoid()
       
        out = identity * a_h * a_w
       
        return out

# Main model with Transformer-Enhanced Attention
class TransformerBrainTumorClassifier(nn.Module):
    def __init__(self, num_classes=3):
        super(TransformerBrainTumorClassifier, self).__init__()
       
        # Load pretrained DenseNet and remove final classifier
        densenet = models.densenet121(weights="IMAGENET1K_V1")
        self.features = nn.Sequential(*list(densenet.features.children()))
       
        # Feature channels at different levels (assuming 1024 total features from DenseNet121)
        feature_dim = 1024
        split_dim = feature_dim // 2
       
        # Replace standard attention with transformer blocks
        self.transformer1 = TransformerBlock(split_dim, num_heads=4, dropout=0.1)
        self.transformer2 = TransformerBlock(split_dim, num_heads=4, dropout=0.1)
       
        # Add coordinate attention
        self.coord_attn1 = CoordinateAttention(split_dim, split_dim)
        self.coord_attn2 = CoordinateAttention(split_dim, split_dim)
       
        # Keep SE blocks
        self.se1 = SEBlock(split_dim)
        self.se2 = SEBlock(split_dim)
       
        # FPN modules
        self.fpn1 = FPN(split_dim)
        self.fpn2 = FPN(split_dim)
       
        # Global pooling and classifier
        self.global_pool = nn.AdaptiveAvgPool2d(1)
       
        # Fully connected layers
        self.classifier = nn.Sequential(
            nn.Linear(512, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.3),
            nn.Linear(512, num_classes)
        )
       
    def forward(self, x):
        # Extract features using DenseNet backbone
        features = self.features(x)
       
        # Split features into two parts
        features1, features2 = torch.split(features, features.size(1)//2, dim=1)
       
        # Apply transformer attention
        features1 = self.transformer1(features1)
        features2 = self.transformer2(features2)
       
        # Apply coordinate attention
        features1 = self.coord_attn1(features1)
        features2 = self.coord_attn2(features2)
       
        # Apply SE blocks
        features1 = self.se1(features1)
        features2 = self.se2(features2)
       
        # Apply FPN
        features1 = self.fpn1(features1)
        features2 = self.fpn2(features2)
       
        # Concatenate features
        combined_features = torch.cat([features1, features2], dim=1)
       
        # Global pooling
        pooled = self.global_pool(combined_features)
        pooled = pooled.view(pooled.size(0), -1)
       
        # Classification
        output = self.classifier(pooled)
       
        return output

# Training function with 10 epochs
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=10):
    best_val_acc = 0.0
    history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}
   
    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        print('-' * 10)
       
        # Training phase
        model.train()
        running_loss = 0.0
        running_corrects = 0
       
        for inputs, labels in train_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
           
            # Zero the parameter gradients
            optimizer.zero_grad()
           
            # Forward pass
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)
           
            # Backward pass and optimize
            loss.backward()
            optimizer.step()
           
            # Statistics
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
       
        scheduler.step()
       
        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_acc = running_corrects.double() / len(train_loader.dataset)
       
        history['train_loss'].append(epoch_loss)
        history['train_acc'].append(epoch_acc.cpu().numpy())
       
        print(f'Train Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
       
        # Validation phase
        model.eval()
        val_running_loss = 0.0
        val_running_corrects = 0
       
        # No gradient calculation needed for validation
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs = inputs.to(device)
                labels = labels.to(device)
               
                # Forward pass
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)
               
                # Statistics
                val_running_loss += loss.item() * inputs.size(0)
                val_running_corrects += torch.sum(preds == labels.data)
       
        val_epoch_loss = val_running_loss / len(val_loader.dataset)
        val_epoch_acc = val_running_corrects.double() / len(val_loader.dataset)
       
        history['val_loss'].append(val_epoch_loss)
        history['val_acc'].append(val_epoch_acc.cpu().numpy())
       
        print(f'Val Loss: {val_epoch_loss:.4f} Acc: {val_epoch_acc:.4f}')
       
        # Save best model
        if val_epoch_acc > best_val_acc:
            best_val_acc = val_epoch_acc
            torch.save(model.state_dict(), 'best_transformer_model.pth')
            print(f'New best model saved with accuracy: {val_epoch_acc:.4f}')
       
        print()
   
    # Load best model weights
    model.load_state_dict(torch.load('best_transformer_model.pth'))
    return model, history

# Evaluation function with advanced metrics
def evaluate_model(model, test_loader, num_classes=3):
    model.eval()
    all_preds = []
    all_labels = []
    all_probs = []
   
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs = inputs.to(device)
            outputs = model(inputs)
            probs = F.softmax(outputs, dim=1)
            _, preds = torch.max(outputs, 1)
           
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.numpy())
            all_probs.extend(probs.cpu().numpy())
   
    # Convert to numpy arrays
    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)
    all_probs = np.array(all_probs)
   
    # Calculate metrics
    accuracy = accuracy_score(all_labels, all_preds)
   
    # Confusion Matrix
    cm = confusion_matrix(all_labels, all_preds)
   
    # Classification Report
    report = classification_report(all_labels, all_preds)
   
    # ROC Curve and AUC
    # Binarize the labels for ROC calculation
    labels_bin = label_binarize(all_labels, classes=range(num_classes))
   
    # Compute ROC curve and ROC area for each class
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(num_classes):
        fpr[i], tpr[i], _ = roc_curve(labels_bin[:, i], all_probs[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])
   
    # Compute micro-average ROC curve and ROC area
    fpr["micro"], tpr["micro"], _ = roc_curve(labels_bin.ravel(), all_probs.ravel())
    roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])
   
    # Compute macro-average ROC curve and ROC area
    # First aggregate all false positive rates
    all_fpr = np.unique(np.concatenate([fpr[i] for i in range(num_classes)]))
   
    # Then interpolate all ROC curves at these points
    mean_tpr = np.zeros_like(all_fpr)
    for i in range(num_classes):
        mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])
   
    # Finally average it and compute AUC
    mean_tpr /= num_classes
   
    fpr["macro"] = all_fpr
    tpr["macro"] = mean_tpr
    roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])
   
    # Plot ROC Curves
    plt.figure(figsize=(12, 8))
   
    # Plot micro-average ROC curve
    plt.plot(fpr["micro"], tpr["micro"],
             label=f'Micro-average ROC curve (area = {roc_auc["micro"]:.2f})',
             color='deeppink', linestyle=':', linewidth=4)
   
    # Plot macro-average ROC curve
    plt.plot(fpr["macro"], tpr["macro"],
             label=f'Macro-average ROC curve (area = {roc_auc["macro"]:.2f})',
             color='navy', linestyle=':', linewidth=4)
   
    # Plot ROC curves for all classes
    colors = cycle(['aqua', 'darkorange', 'cornflowerblue'])
    class_names = ['Meningioma', 'Glioma', 'Pituitary']
   
    for i, color, name in zip(range(num_classes), colors, class_names):
        plt.plot(fpr[i], tpr[i], color=color, lw=2,
                 label=f'ROC curve of {name} (area = {roc_auc[i]:.2f})')
   
    plt.plot([0, 1], [0, 1], 'k--', lw=2)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend(loc="lower right")
    plt.savefig('transformer_roc_curve.png')
    plt.close()
   
    # Plot Confusion Matrix
    plt.figure(figsize=(10, 8))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title('Confusion Matrix')
    plt.colorbar()
   
    tick_marks = np.arange(num_classes)
    plt.xticks(tick_marks, class_names, rotation=45)
    plt.yticks(tick_marks, class_names)
   
    # Add text annotations to the confusion matrix
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            plt.text(j, i, format(cm[i, j], 'd'),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
   
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()
    plt.savefig('transformer_confusion_matrix.png')
    plt.close()
   
    # Return metrics and predictions
    return {
        'accuracy': accuracy,
        'confusion_matrix': cm,
        'classification_report': report,
        'roc_auc': roc_auc,
        'predictions': all_preds,
        'labels': all_labels,
        'probabilities': all_probs
    }

# Function to test with a random image
def test_random_image(model, images, labels, transform=None):
    """
    Test the model with a random image from the dataset
   
    Args:
        model: Trained model
        images: Image data
        labels: Labels
        transform: Image transform function
       
    Returns:
        prediction: The model's prediction
    """
    # Choose a random image
    idx = random.randint(0, len(images) - 1)
    img = images[idx]
    true_label = labels[idx] - 1  # Convert to 0-indexed
   
    # Preprocess the image
    img = np.repeat(img.reshape(224, 224, 1), 3, axis=2)
   
    # Apply transform if provided
    if transform:
        img_tensor = transform(img).unsqueeze(0).to(device)  # Add batch dimension
    else:
        img_tensor = torch.from_numpy(img.transpose(2, 0, 1)).float().unsqueeze(0).to(device)
   
    # Get model prediction
    model.eval()
    with torch.no_grad():
        output = model(img_tensor)
        probs = F.softmax(output, dim=1)
        _, pred = torch.max(output, 1)
   
    # Display the image and prediction
    class_names = ['Meningioma', 'Glioma', 'Pituitary']
    plt.figure(figsize=(6, 6))
    plt.imshow(img, cmap='gray')
    plt.title(f'True: {class_names[true_label]}, Pred: {class_names[pred.item()]}\nConfidence: {probs[0][pred.item()]:.4f}')
    plt.axis('off')
    plt.savefig('transformer_random_test.png')
    plt.close()
   
    print(f"Random image test - True: {class_names[true_label]}, Predicted: {class_names[pred.item()]}")
    print(f"Confidence scores: {probs[0].cpu().numpy()}")
   
    return {
        'image_idx': idx,
        'true_label': true_label,
        'predicted_label': pred.item(),
        'confidence': probs[0][pred.item()].item(),
        'all_probs': probs[0].cpu().numpy()
    }

# Main execution function
def run_experiment(fold_index):
    print(f"\n{'='*20} RUNNING FOLD {fold_index} {'='*20}\n")
   
    # Load data
    images, labels, fold_indices = load_data()
   
    # Get train and test data for this fold
    (train_images, train_labels), (test_images, test_labels) = get_train_test_data(
        images, labels, fold_indices, fold_index)
   
    # Create datasets with transforms
    train_transform, val_transform = get_transforms()
   
    train_dataset = BrainTumorDataset(train_images, train_labels, transform=train_transform)
    test_dataset = BrainTumorDataset(test_images, test_labels, transform=val_transform)
   
    # Create data loaders
    train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=4)
    test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=4)
   
    # Initialize model with Transformer-Enhanced architecture
    model = TransformerBrainTumorClassifier(num_classes=3)
    model = model.to(device)
   
    # Loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)
   
    # Set scheduler with T_max=10 to match epochs
    scheduler = CosineAnnealingLR(optimizer, T_max=10, eta_min=1e-6)
   
    # Train model with 10 epochs
    start_time = time.time()
    model, history = train_model(
        model, train_loader, test_loader, criterion, optimizer, scheduler, num_epochs=10)
    end_time = time.time()
   
    # Evaluate model with advanced metrics
    metrics = evaluate_model(model, test_loader)
    accuracy = metrics['accuracy']
   
    print(f"Fold {fold_index} accuracy: {accuracy:.4f}")
    print(f"Training time: {end_time - start_time:.2f} seconds")
    print("\nClassification Report:")
    print(metrics['classification_report'])
   
    # Get ROC AUC scores
    print("\nROC AUC Scores:")
    for i in range(3):
        print(f"Class {i}: {metrics['roc_auc'][i]:.4f}")
    print(f"Micro-average: {metrics['roc_auc']['micro']:.4f}")
    print(f"Macro-average: {metrics['roc_auc']['macro']:.4f}")
   
    # Plot loss curve
    plt.figure(figsize=(10, 5))
    plt.plot(history['train_loss'], label='Train Loss')
    plt.plot(history['val_loss'], label='Val Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.title(f'Loss Curves for Fold {fold_index} with Transformer')
    plt.savefig(f'transformer_loss_curve_fold_{fold_index}.png')
    plt.close()
   
    # Test with a random image
    random_test_results = test_random_image(model, images, labels, transform=val_transform)
   
    # Save model and results
    torch.save(model.state_dict(), f'transformer_model_fold_{fold_index}.pth')
    np.save(f'transformer_results_fold_{fold_index}.npy', metrics)
   
    return metrics

# Run for all folds
if __name__ == "__main__":
    results_all = {}
   
    # Run for all 5 folds
    for fold in range(1, 6):
        try:
            metrics = run_experiment(fold)
            results_all[fold] = metrics
        except Exception as e:
            print(f"Error in fold {fold}: {e}")
   
    # Calculate and print average accuracy across all folds
    accuracies = [results_all[fold]['accuracy'] for fold in results_all]
    if accuracies:
        avg_accuracy = np.mean(accuracies)
        print(f"\nAverage accuracy across all folds: {avg_accuracy:.4f}")
       
        # Calculate average AUC across all folds
        avg_auc_micro = np.mean([results_all[fold]['roc_auc']['micro'] for fold in results_all])
        avg_auc_macro = np.mean([results_all[fold]['roc_auc']['macro'] for fold in results_all])
        print(f"Average micro-average AUC: {avg_auc_micro:.4f}")
        print(f"Average macro-average AUC: {avg_auc_macro:.4f}")

Using device: cuda
Path to dataset files: /kaggle/input/figshare-brain-tumor-dataset

==================== RUNNING FOLD 1 ====================

Found 3064 .mat files
Processed 0 images


/tmp/ipykernel_83/321735910.py:65: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  lbl.append(int(images['label'][0]))


Processed 500 images
Processed 1000 images
Processed 1500 images
Processed 2000 images
Processed 2500 images
Processed 3000 images
Found cvind.mat at: /kaggle/input/figshare-brain-tumor-dataset/dataset/cvind.mat


Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth
100%|██████████| 30.8M/30.8M [00:00<00:00, 153MB/s] 


Epoch 1/10
----------
Train Loss: 0.3406 Acc: 0.8676
Val Loss: 0.1878 Acc: 0.9354
New best model saved with accuracy: 0.9354

Epoch 2/10
----------
Train Loss: 0.1984 Acc: 0.9306
Val Loss: 0.1952 Acc: 0.9299

Epoch 3/10
----------
Train Loss: 0.1584 Acc: 0.9500
Val Loss: 0.0787 Acc: 0.9760
New best model saved with accuracy: 0.9760

Epoch 4/10
----------
Train Loss: 0.0994 Acc: 0.9651
Val Loss: 0.1057 Acc: 0.9649

Epoch 5/10
----------
Train Loss: 0.0853 Acc: 0.9715
Val Loss: 0.0992 Acc: 0.9668

Epoch 6/10
----------
Train Loss: 0.0672 Acc: 0.9802
Val Loss: 0.0758 Acc: 0.9723

Epoch 7/10
----------
Train Loss: 0.0697 Acc: 0.9806
Val Loss: 0.0734 Acc: 0.9760

Epoch 8/10
----------
Train Loss: 0.0353 Acc: 0.9913
Val Loss: 0.0617 Acc: 0.9760

Epoch 9/10
----------
Train Loss: 0.0264 Acc: 0.9941
Val Loss: 0.0526 Acc: 0.9797
New best model saved with accuracy: 0.9797

Epoch 10/10
----------
Train Loss: 0.0254 Acc: 0.9925
Val Loss: 0.0518 Acc: 0.9834
New best model saved with accuracy: 0.983

...

Fold 4 accuracy: 0.9809
Training time: 307.96 seconds

Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.96      0.97       168
           1       0.98      0.98      0.98       287
           2       0.98      1.00      0.99       173

    accuracy                           0.98       628
   macro avg       0.98      0.98      0.98       628
weighted avg       0.98      0.98      0.98       628


ROC AUC Scores:
Class 0: 0.9988
Class 1: 0.9992
Class 2: 0.9999
Micro-average: 0.9994
Macro-average: 0.9994
Random image test - True: Pituitary, Predicted: Pituitary
Confidence scores: [1.6409383e-04 5.1482652e-06 9.9983072e-01]

==================== RUNNING FOLD 5 ====================

Found 3064 .mat files
Processed 0 images


/tmp/ipykernel_83/321735910.py:65: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  lbl.append(int(images['label'][0]))


Processed 500 images
Processed 1000 images
Processed 1500 images
Processed 2000 images
Processed 2500 images
Processed 3000 images
Found cvind.mat at: /kaggle/input/figshare-brain-tumor-dataset/dataset/cvind.mat
Epoch 1/10
----------
Train Loss: 0.3352 Acc: 0.8769
Val Loss: 0.2275 Acc: 0.9176
New best model saved with accuracy: 0.9176

Epoch 2/10
----------
Train Loss: 0.1955 Acc: 0.9294
Val Loss: 0.1333 Acc: 0.9565
New best model saved with accuracy: 0.9565

Epoch 3/10
----------
Train Loss: 0.1655 Acc: 0.9434
Val Loss: 0.1275 Acc: 0.9549

Epoch 4/10
----------
Train Loss: 0.1004 Acc: 0.9632
Val Loss: 0.1173 Acc: 0.9627
New best model saved with accuracy: 0.9627

Epoch 5/10
----------
Train Loss: 0.0770 Acc: 0.9773
Val Loss: 0.1285 Acc: 0.9502

Epoch 6/10
----------
Train Loss: 0.0600 Acc: 0.9806
Val Loss: 0.0985 Acc: 0.9689
New best model saved with accuracy: 0.9689

Epoch 7/10
----------
Train Loss: 0.0665 Acc: 0.9831
Val Loss: 0.0665 Acc: 0.9736
New best model saved with accuracy: 

/tmp/ipykernel_83/321735910.py:404: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_transformer_model.pth'))


Fold 5 accuracy: 0.9782
Training time: 306.54 seconds

Classification Report:
              precision    recall  f1-score   support

           0       0.95      0.96      0.96       144
           1       0.99      0.98      0.99       296
           2       0.98      0.99      0.98       203

    accuracy                           0.98       643
   macro avg       0.97      0.98      0.97       643
weighted avg       0.98      0.98      0.98       643


ROC AUC Scores:
Class 0: 0.9965
Class 1: 0.9995
Class 2: 0.9985
Micro-average: 0.9985
Macro-average: 0.9985
Random image test - True: Pituitary, Predicted: Pituitary
Confidence scores: [8.1296668e-05 3.9397471e-04 9.9952483e-01]

Average accuracy across all folds: 0.9830
Average micro-average AUC: 0.9987
Average macro-average AUC: 0.9991
